In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from shapely.geometry import Polygon, Point
import sys
import os
import statistics
import psutil
import copy
import pickle
import datetime
from datetime import datetime
import re
from abc import ABC, abstractmethod
import math
import copy
import xml.etree.ElementTree as ET
import gc
from copy import deepcopy
from pathlib import Path
from skimage import data
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import skimage
import PIL
from skimage.filters import threshold_otsu
import torchvision.models as torchmodels
import openslide
import torch.utils.data
import torchvision
import torch.nn as nn
list_pathstoadd = ["../../../",                   "../../../Src/BoVWPipeline/",
                   "../../../../PyDmed/"]
for path in list_pathstoadd:
    if(path not in sys.path):
        sys.path.append(path)
import pydmed
from pydmed.utils.data import *
import pydmed.lightdl
from pydmed.lightdl import *
import pydmed.stat
from pydmed.stat import *
import pydmed.extensions.dl
import projutils.datasets
import relatedwork
from relatedwork.utils.generativemodels import ResidualEncoder
import bovw_pipeline
from bovw_pipeline import *
import localsrc_DA
from localsrc_DA import *


#SETTINGS ================
idx_split = 1
num_visualwords = 10
lr_optimizer = 0.000005
num_classes = 4
batchsize = 32
stepsize_gradupdate = 20
num_warmup_droppedpatches = 10
scale_thumbnail = 0.005
val_randomseed = 15



In [2]:
mode_trainortest = "train"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


# # Make Datasets (train/val/test)

#build datasets ==============================================
tgt_train_ds, tgt_val_ds, tgt_test_ds = projutils.datasets.get_warwickher2(
             idx_split=idx_split,
             scale_foreground=scale_thumbnail
            )
tgt_list_allpatients = tgt_train_ds.list_patients + tgt_val_ds.list_patients +  tgt_test_ds.list_patients



#build datasets ==============================================
src_train_ds, src_val_ds, src_test_ds = projutils.datasets.get_privateher2(
             idx_split=idx_split,
             scale_foreground=scale_thumbnail
            )
src_list_allpatients = src_train_ds.list_patients + src_val_ds.list_patients + src_test_ds.list_patients


cuda:0
annotation 0
[['54301.6406' '8.42716503']
 ['55424.4492' '58611.2383']
 ['118773.539' '58607']
 ['118781.008' '3.34055328']]




annotation 0
[['50077.6445' '3.68681288']
 ['51752.0508' '81148.6172']
 ['131069.133' '81148.8594']
 ['131069.281' '2.666435']]




annotation 0
[['1.95234442' '2.23125076']
 ['1.68418598' '43517.6797']
 ['49149.6719' '43518.0117']
 ['49150.0273' '1.96491194']]




annotation 0
[['70693.4531' '4.307477']
 ['71314.3594' '51960.9297']
 ['143355.359' '51963.5508']
 ['143356.109' '2.99877715']]




annotation 0
[['6.10493279' '8.13990974']
 ['5.59031487' '61432.9844']
 ['127227.516' '61428.9609']
 ['120928.852' '5.32723618']]




annotation 0
[['69712.4688' '28.2122478']
 ['70098.1953' '31991.7461']
 ['122874.164' '31995.3516']
 ['122874.867' '4.19429016']]




annotation 0
[['46168.3945' '334.207611']
 ['46437.3555' '39673.5039']
 ['122874.188' '39675.4375']
 ['122877.055' '2.60024929']]




annotation 0
[['61620.6523' '459.284363']
 ['61737.1367' '46075.

 computing foreground for patient 37
 computing foreground for patient 38
 computing foreground for patient 39
 computing foreground for patient 40
 computing foreground for patient 41
 computing foreground for patient 42
 computing foreground for patient 43
 computing foreground for patient 44
 computing foreground for patient 45
 computing foreground for patient 46
 computing foreground for patient 47
 computing foreground for patient 48
 computing foreground for patient 49
 computing foreground for patient 50
 computing foreground for patient 51
elapsed time = 2.235835313796997
annotation 0
[['84235.875' '8.46127796']
 ['84226.125' '93038.2031']
 ['159315.266' '92982.8828']
 ['159466.344' '17.9788418']]




annotation 0
[['41981.875' '332.435669']
 ['42623.5117' '55762.793']
 ['141493.109' '56716.168']
 ['141292.422' '104.73864']]




annotation 0
[['64980.4727' '410.402985']
 ['64433.2695' '46785.9414']
 ['133380.969' '46785.9414']
 ['133510.062' '133.310089']]




annotation 0
[['

annotation 0
[['54525.4609' '421.046021']
 ['55999.1172' '23368.0527']
 ['71472.5625' '59367.4883']
 ['121577.031' '59472.75']
 ['121366.508' '315.784515']]




annotation 0
[['71171.0859' '322.041107']
 ['52895.2539' '42589.9375']
 ['55254.7539' '64330.9805']
 ['109636.203' '64524.9336']
 ['109643.188' '17.9189987']]




annotation 0
[['48239.4766' '1550.55469']
 ['47894.9102' '62711.3203']
 ['117325.297' '63400.457']
 ['117332.344' '370.133575']]




annotation 0
[['45063.1367' '14.8567476']
 ['45900.8828' '52688.1445']
 ['115621.195' '52721.4375']
 ['115617.992' '12.1577911']]




annotation 0
[['43552.8633' '160.199356']
 ['50291.9258' '56240.082']
 ['101575.133' '56352.7734']
 ['101660.5' '15.814827']]




annotation 0
[['56918.5' '374.463806']
 ['55982.3398' '48118.6016']
 ['104662.633' '46995.207']
 ['105130.719' '187.231903']]




annotation 0
[['5.42111731' '6.09875679']
 ['7.47310877' '26612.418']
 ['29897.6309' '26619.0547']
 ['29895.9434' '6.11813831']]




annotation 0
[['

annotation 0
[['59087.5586' '761.192383']
 ['80971.8438' '66223.7344']
 ['129573.523' '66646.9297']
 ['129480.5' '54.1831551']]




annotation 0
[['77097.875' '48319.8555']
 ['88375.75' '777.784424']
 ['95375.8125' '2333.35327']
 ['83806.2656' '50167.0938']]




annotation 0
[['36486.0469' '1324.09033']
 ['46675.25' '85868.4297']
 ['159346.203' '85776.9609']
 ['159434.875' '44.3402519']]




annotation 0
[['70151.0078' '1749.40161']
 ['70325.9453' '51782.2891']
 ['119134.25' '52307.1094']
 ['119309.195' '524.820496']]




annotation 0
[['50773.1602' '114.073677']
 ['39845.1875' '70246.2188']
 ['115518.734' '70272.2578']
 ['115569.57' '32.6533623']]




annotation 0
[['106111.742' '8548.78125']
 ['84205.4922' '51185.8242']
 ['98314.7891' '64704.4258']
 ['145510.141' '64862.7383']
 ['144179.047' '9552.9668']]




annotation 0
[['31716.8867' '80.2282104']
 ['33193.3516' '79521.8828']
 ['147296.094' '79330.3047']
 ['147329.922' '288.060608']]




annotation 0
[['70674.9688' '795.377502']
 

annotation 0
[['67751.9141' '445.004395']
 ['77692.1875' '63818.582']
 ['151294.609' '64847.5586']
 ['151376.156' '129.451614']]




annotation 0
[['99779.2031' '123.303131']
 ['88874.9453' '29624.9824']
 ['123308.492' '43694.1719']
 ['123593.219' '17.3270836']]




annotation 0
[['71837.5312' '1712.68286']
 ['71227.8203' '70310.3672']
 ['129544.727' '70811.1484']
 ['129413.891' '124.245293']]




annotation 0
[['39750.8086' '94.1070251']
 ['43517.5117' '56596.8359']
 ['101631.102' '56518.1094']
 ['101661.977' '17.7386341']]




annotation 0
[['65055.7891' '60.9564667']
 ['67042.6641' '67521.0156']
 ['143540.016' '67617.5469']
 ['143531.328' '18.9811211']]




annotation 0
[['66213.3984' '931.853943']
 ['66938.1719' '18274.6914']
 ['94065.4766' '19776.0117']
 ['104298.93' '35082.4023']
 ['125595.359' '35082.9961']
 ['125593.984' '10.9278183']]




annotation 0
[['38917.0742' '14.2982073']
 ['55790.0078' '63896.5859']
 ['141542.922' '63885.5859']
 ['141548.375' '7.91955805']]




annota

...

annotation 0
[['12.6086788' '14.7101259']
 ['210.495163' '42249.3867']
 ['75751.4766' '42499.3281']
 ['75755.3203' '5.52686262']]




annotation 0
[['50240.0664' '1542.45825']
 ['41205.668' '53765.6875']
 ['45169.0859' '88039.1484']
 ['149350.359' '88341.9375']
 ['148694.344' '656.004456']]




annotation 0
[['65652.8672' '808.035278']
 ['69289.0234' '69895.0469']
 ['136658.969' '70501.0781']
 ['137302.891' '356.86261']]




annotation 0
[['138024.969' '2254.30518']
 ['97652.4062' '21825.7734']
 ['98592.1797' '53455.2734']
 ['139544.891' '53695.0508']]




annotation 0
[['57383.1914' '224.551804']
 ['58813.8047' '56280.6445']
 ['119605.828' '56284.9336']
 ['119613.406' '10.2083769']]




annotation 0
[['56209.3477' '100.486427']
 ['53239.2891' '58616.1016']
 ['123578.992' '58851.7227']
 ['123596.523' '16.5705128']]




annotation 0
[['48442.332' '596.825378']
 ['49672.543' '47322.918']
 ['123562.023' '47650.5039']
 ['123486.805' '176.158066']]




annotation 0
[['29565.3457' '53.697418

annotation 0
[['56315.4062' '11.6627817']
 ['61203.7188' '50424.2266']
 ['117607.391' '50443.0508']
 ['117593.219' '31.3105125']]




annotation 0
[['45553.8789' '2237.02075']
 ['46774.0703' '64263.5078']
 ['136661.641' '64263.5078']
 ['137068.359' '1016.82764']]




annotation 0
[['37490.6172' '124.474777']
 ['40457.4375' '66752.25']
 ['125565.258' '66932.8828']
 ['125484.359' '135.1306']]




annotation 0
[['45672.7539' '787.461243']
 ['44869.9102' '62056.0156']
 ['107471.648' '62646.8789']
 ['107562.945' '75.6081772']]




annotation 0
[['41088.7109' '282.396637']
 ['44031.0625' '62793.6406']
 ['111440.414' '62452.5234']
 ['111474.594' '142.616943']]




annotation 0
[['127581.086' '1235.65222']
 ['90665.9766' '33362.6094']
 ['93600.6484' '45255.7617']
 ['111980.977' '46182.5']
 ['140709.891' '17762.5']
 ['141018.797' '772.282593']]




annotation 0
[['118013' '271.071716']
 ['107321.984' '64702.6406']
 ['150882.078' '61746.4531']
 ['151514.828' '9.03092384']]




annotation 0
[['51

annotation 0
[['77423.6016' '1467.28247']
 ['76984.75' '59983.9531']
 ['121605.367' '46202.6016']
 ['121613.477' '7.41274357']]




annotation 0
[['40809.9727' '48.3673744']
 ['43814.1211' '71421.0078']
 ['119516.992' '71445.7969']
 ['119614.367' '12.9969387']]




annotation 0
[['26204.5801' '67.8401947']
 ['28822.8789' '85585.3203']
 ['157350.516' '85488.8203']
 ['157200.391' '159.526382']]




annotation 0
[['90319.0938' '21266.0977']
 ['95897.6953' '48874.0156']
 ['129533.828' '48931.2305']
 ['129358.242' '10538.2363']]




annotation 0
[['42651.5469' '1646.20007']
 ['35767.4375' '88595.4922']
 ['147409.734' '88745.1484']
 ['147110.422' '598.618164']]




annotation 0
[['47008.9727' '22.4167652']
 ['51011.4883' '61438.0273']
 ['165460.562' '61473.7695']
 ['165474.062' '6.10170078']]




annotation 0
[['51550.1289' '415.726868']
 ['53553.2891' '72034.0234']
 ['140603.141' '72162.2031']
 ['140914.531' '98.5760956']]




annotation 0
[['59421.4414' '34.8134155']
 ['60638.0352' '67922.

annotation 0
[['33854.9297' '158.885956']
 ['41198.7344' '72169.4141']
 ['136031.391' '72111.4297']
 ['152762.453' '24.542942']]




annotation 0
[['36772.1914' '70.0689621']
 ['37820.9766' '55770.2461']
 ['103574.867' '55693.9766']
 ['103610.312' '95.9523926']]




annotation 0
[['54646.2461' '82.3233643']
 ['63777.3125' '58204.5352']
 ['129572.508' '58384.8125']
 ['129571.57' '15.484868']]




annotation 0
[['56767.5078' '270.505463']
 ['57702.4531' '62228.3906']
 ['125544.82' '62232.7305']
 ['125559.32' '58.261837']]




annotation 0
[['34489.0547' '340.352509']
 ['101084.695' '680.705017']
 ['100857.797' '53435.3477']
 ['35283.2109' '53548.7969']]




annotation 0
[['30897.959' '296.739105']
 ['42273.1914' '87752.4531']
 ['143536.062' '87852.7656']
 ['143542.094' '3.63041091']]




annotation 0
[['87131.375' '237.05632']
 ['80907.7188' '64738.5469']
 ['169448.797' '65036.207']
 ['169458.656' '7.51703405']]




annotation 0
[['103447.492' '538.228394']
 ['85793.6016' '31647.8281']
 

annotation 0
[['88325.3828' '432.96759']
 ['45353.3555' '45136.8711']
 ['50116' '57584.6914']
 ['106942.992' '57584.6914']
 ['106942.992' '649.451416']]




annotation 0
[['90110.1562' '1031.00867']
 ['91759.7734' '34641.8906']
 ['118978.406' '50107.0234']
 ['138361.375' '22888.3926']
 ['138009.844' '386.582184']]




annotation 0
[['74103.9688' '890.137756']
 ['42949.1484' '20473.168']
 ['53630.8008' '68095.5391']
 ['150655.812' '68095.5391']
 ['150210.75' '667.603333']]




annotation 0
[['44746.457' '12.4953804']
 ['48481.4336' '60963.2148']
 ['135561.297' '60982.3359']
 ['135568.312' '4.95362425']]




annotation 0
[['52075.3555' '8.37318516']
 ['49525.9766' '59479.3125']
 ['73752.5312' '59491.4102']
 ['73759.9688' '4.15816164']]




elapsed time = 7.152557373046875e-07


In [3]:
tgt_tfms = torchvision.transforms.ToTensor()
tgt_const_global_info = {
    "mode_trainortest": mode_trainortest,
    "width_bigchunk":2000,
    "heigth_bigchunk":2000,
    "width_smallchunk":224,
    "heigth_smallchunk":224,
    "num_bigchunkloaders":10,
    "maxlength_queue_smallchunk":100,
    "maxlength_queue_lightdl":2000,
    "interval_resched": 10,
    "core-assignment":{"lightdl":None,
                       "smallchunkloaders":None,
                       "bigchunkloaders":None}
}

src_tfms = torchvision.transforms.ToTensor()
src_const_global_info = {
    "mode_trainortest": mode_trainortest,
    "width_bigchunk":2000,
    "heigth_bigchunk":2000,
    "width_smallchunk":224,
    "heigth_smallchunk":224,
    "num_bigchunkloaders":10,
    "maxlength_queue_smallchunk":100,
    "maxlength_queue_lightdl":2000,
    "interval_resched": 10,
    "core-assignment":{"lightdl":None,
                       "smallchunkloaders":None,
                       "bigchunkloaders":None}
}

In [4]:
def tgt_func_getlabel_of_patient(patient):
    return patient.dict_records["Her2score"]

tgt_train_dl = pydmed.extensions.dl.LabelBalancedDL(
                  func_getlabel_of_patient = tgt_func_getlabel_of_patient,\
                  dataset=tgt_train_ds,\
                  type_bigchunkloader=WSIRandomBigchunkLoader,\
                  type_smallchunkcollector=WSIRandomSmallchunkCollector,\
                  const_global_info=tgt_const_global_info,\
                  batch_size=batchsize, tfms=tgt_tfms, fname_logfile=None,
                  flag_grabqueue_onunsched = False
                 )

print("The src_dataloaders created.")


#subsample the original training set =====
def src_func_getlabel_of_patient(patient):
    return patient.dict_records["Her2_score"]

src_train_count=[43,119,63,25]

src_train_new_list_patients = []
for patient in src_train_ds.list_patients:
    if src_train_count[patient.dict_records['Her2_score']]>0:
        src_train_count[patient.dict_records['Her2_score']] -=1
        src_train_new_list_patients.append(patient)
print(src_train_count)
        
src_test_count=[43,119,63,25]
src_test_new_list_patients = []
for patient in src_test_ds.list_patients:
    if src_test_count[patient.dict_records['Her2_score']]>0:
        src_test_count[patient.dict_records['Her2_score']] -=1
        src_test_new_list_patients.append(patient)
print(src_test_count)
        
print(len(src_train_new_list_patients))
print(len(src_test_new_list_patients))

projutils.datasets.get_segmentation(src_train_new_list_patients, scale_foreground=scale_thumbnail)

src_train_ds = pydmed.utils.data.Dataset("train_private", src_train_new_list_patients)


src_train_dl = pydmed.extensions.dl.LabelBalancedDL(
                  func_getlabel_of_patient = src_func_getlabel_of_patient,\
                  dataset=src_train_ds,\
                  type_bigchunkloader=WSIRandomBigchunkLoader,\
                  type_smallchunkcollector=WSIRandomSmallchunkCollector,\
                  const_global_info=src_const_global_info,\
                  batch_size=batchsize, tfms=src_tfms, fname_logfile=None,
                  flag_grabqueue_onunsched = False
                 )
print("The tgt_dataloaders created.")




The src_dataloaders created.
[0, 0, 0, 0]
[0, 0, 0, 0]
250
250
 computing foreground for patient 0
 computing foreground for patient 1
 computing foreground for patient 2
 computing foreground for patient 3
 computing foreground for patient 4
 computing foreground for patient 5
 computing foreground for patient 6
 computing foreground for patient 7
 computing foreground for patient 8
 computing foreground for patient 9
 computing foreground for patient 10
 computing foreground for patient 11
 computing foreground for patient 12
 computing foreground for patient 13
 computing foreground for patient 14
 computing foreground for patient 15
 computing foreground for patient 16
 computing foreground for patient 17
 computing foreground for patient 18
 computing foreground for patient 19
 computing foreground for patient 20
 computing foreground for patient 21
 computing foreground for patient 22
 computing foreground for patient 23
 computing foreground for patient 24
 computing foreground 

 computing foreground for patient 217
 computing foreground for patient 218
 computing foreground for patient 219
 computing foreground for patient 220
 computing foreground for patient 221
 computing foreground for patient 222
 computing foreground for patient 223
 computing foreground for patient 224
 computing foreground for patient 225
 computing foreground for patient 226
 computing foreground for patient 227
 computing foreground for patient 228
 computing foreground for patient 229
 computing foreground for patient 230
 computing foreground for patient 231
 computing foreground for patient 232
 computing foreground for patient 233
 computing foreground for patient 234
 computing foreground for patient 235
 computing foreground for patient 236
 computing foreground for patient 237
 computing foreground for patient 238
 computing foreground for patient 239
 computing foreground for patient 240
 computing foreground for patient 241
 computing foreground for patient 242
 computing f

In [5]:
#build the model and optimizer====================
model = localsrc_DA.Pipeline_DA(num_classes=num_classes, num_visualwords=num_visualwords,   device_stg3=device, size_input=[batchsize,3,224,224])
optimizer = torch.optim.Adam(model.parameters(), lr=lr_optimizer)
criterion = nn.CrossEntropyLoss(
                    weight=torch.from_numpy(np.array([1.0,1.0,1.0,1.0]))
                ).to(device).float()
model.to(device)
print("")


In [6]:
#train the model ============================
if('history_trainingloss' not in globals()):
    history_trainingloss = []
    history_testloss = []
    history_fed_labels = []
    list_seenbatches = []
    
src_train_dl.start()
tgt_train_dl.start()
time.sleep(30)

print("========== pid of the root process = {} ============".format(os.getpid()))
model.train()
tstart = time.time()
batchcount = 0
optimizer.zero_grad()

 loading initial bigchunks, please wait ....
     bigchunk 0 from 10, please wait ...

 loading initial bigchunks, please wait ....
     bigchunk 0 from 10, please wait ...

     bigchunk 1 from 10, please wait ...

     bigchunk 1 from 10, please wait ...

     bigchunk 2 from 10, please wait ...

     bigchunk 2 from 10, please wait ...

     bigchunk 3 from 10, please wait ...

========== pid of the root process = 5867 ============
     bigchunk 4 from 10, please wait ...

     bigchunk 5 from 10, please wait ...

     bigchunk 3 from 10, please wait ...

     bigchunk 6 from 10, please wait ...

     bigchunk 7 from 10, please wait ...

     bigchunk 8 from 10, please wait ...

     bigchunk 4 from 10, please wait ...

     bigchunk 9 from 10, please wait ...

The initial loading of bigchunks took 93.27834415435791 seconds.
     bigchunk 5 from 10, please wait ...

     bigchunk 6 from 10, please wait ...

     bigchunk 7 from 10, please wait ...

     bigchunk 8 from 10, please wa

Process WSIRandomBigchunkLoader-1:675:1:
Traceback (most recent call last):
  File "/home/microscopy-imaging/Desktop/OnGit/uda_and_microscopyimaging_repo2/Runs/IPMI2021_BoVW/DA_warwick_private/localsrc_DA.py", line 190, in extract_bigchunk
    pil_bigchunk = osimage.read_region([rand_x, rand_y], 1, [w,h])
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/openslide/__init__.py", line 229, in read_region
    level, size[0], size[1])
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/openslide/lowlevel.py", line 214, in read_region
    _read_region(slide, buf, x, y, level, w, h)
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/openslide/lowlevel.py", line 151, in _check_error
    raise OpenSlideError(err)
openslide.lowlevel.OpenSlideError: Restart marker not found at recorded position 341572847

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/microscopy-imag

failed for patient utils.data.Patient with unique id: 1214001
failed for patient utils.data.Patient with unique id: 905001
failed for patient utils.data.Patient with unique id: 746001
failed for patient utils.data.Patient with unique id: 484001


Process WSIRandomBigchunkLoader-2:663:1:
Traceback (most recent call last):
Process WSIRandomBigchunkLoader-2:664:1:
Process WSIRandomBigchunkLoader-2:665:1:
Process WSIRandomBigchunkLoader-2:667:1:
  File "/home/microscopy-imaging/Desktop/OnGit/uda_and_microscopyimaging_repo2/Runs/IPMI2021_BoVW/DA_warwick_private/localsrc_DA.py", line 190, in extract_bigchunk
    pil_bigchunk = osimage.read_region([rand_x, rand_y], 1, [w,h])
Traceback (most recent call last):


failed for patient utils.data.Patient with unique id: 637001


Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/openslide/__init__.py", line 229, in read_region
    level, size[0], size[1])
Process WSIRandomBigchunkLoader-2:669:1:
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/openslide/lowlevel.py", line 214, in read_region
    _read_region(slide, buf, x, y, level, w, h)
  File "/home/microscopy-imaging/Desktop/OnGit/uda_and_microscopyimaging_repo2/Runs/IPMI2021_BoVW/DA_warwick_private/localsrc_DA.py", line 182, in extract_bigchunk
    osimage = openslide.OpenSlide(fname_wsi)
  File "/home/microscopy-imaging/Desktop/OnGit/uda_and_microscopyimaging_repo2/Runs/IPMI2021_BoVW/DA_warwick_private/localsrc_DA.py", line 190, in extract_bigchunk
    pil_bigchunk = osimage.read_region([rand_x, rand_y], 1, [w,h])
  File "/home/microscopy-imaging/Desktop/OnGit/uda_and_microscopyimaging_repo2/Runs/IPMI2021_BoVW/DA_warwick_private/localsrc_DA.py"

failed for patient utils.data.Patient with unique id: 8000


Process WSIRandomBigchunkLoader-1:676:1:
Traceback (most recent call last):
  File "/home/microscopy-imaging/Desktop/OnGit/uda_and_microscopyimaging_repo2/Runs/IPMI2021_BoVW/DA_warwick_private/localsrc_DA.py", line 182, in extract_bigchunk
    osimage = openslide.OpenSlide(fname_wsi)
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/openslide/__init__.py", line 160, in __init__
    self._osr = lowlevel.open(filename)
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/openslide/lowlevel.py", line 129, in _check_open
    "Unsupported or missing image file")
openslide.lowlevel.OpenSlideUnsupportedFormatError: Unsupported or missing image file

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/microscopy-imaging/Desktop/OnGit/uda_and_microscopyimaging_repo2/Ru

failed for patient utils.data.Patient with unique id: 167001


Process WSIRandomBigchunkLoader-2:670:1:
Traceback (most recent call last):
  File "/home/microscopy-imaging/Desktop/OnGit/uda_and_microscopyimaging_repo2/Runs/IPMI2021_BoVW/DA_warwick_private/localsrc_DA.py", line 182, in extract_bigchunk
    osimage = openslide.OpenSlide(fname_wsi)
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/openslide/__init__.py", line 160, in __init__
    self._osr = lowlevel.open(filename)
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/openslide/lowlevel.py", line 129, in _check_open
    "Unsupported or missing image file")
openslide.lowlevel.OpenSlideUnsupportedFormatError: Unsupported or missing image file

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/microscopy-imaging/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/microscopy-imaging/Desktop/OnGit/uda_and_microscopyimaging_repo2/Ru

In [7]:
while True:
    #get x and y =========
    src_x, src_list_patients, src_list_smallchunks = src_train_dl.get()
    tgt_x, tgt_list_patients, tgt_list_smallchunks = tgt_train_dl.get()
    
    
    src_tensor_list_assignmentindices, src_list_different_groups = getassignment_img_to_wsi(src_x, 
                                                                                            src_list_patients, 
                                                                                            src_list_smallchunks)
    
    
    tgt_tensor_list_assignmentindices, tgt_list_different_groups = getassignment_img_to_wsi(tgt_x, 
                                                                                            tgt_list_patients, 
                                                                                            tgt_list_smallchunks)
    src_y = [src_func_getlabel_of_patient(patient)  for patient in src_list_different_groups]
    
    
    
    batchcount += 1
    #training step =========
    if((batchcount%10) == 0):
        print(" batchcount {}".format(batchcount))
    
    if(batchcount < num_warmup_droppedpatches):
        print("    the batch dropped (warm-up).")
        continue
    assert(batchcount >= num_warmup_droppedpatches)
    
    src_y = torch.from_numpy(np.array(src_y))
    
    netout, src_DA_output, tgt_DA_output, src_global_DA_output, tgt_global_DA_output = model(src_x.to(device),   
                   tgt_x.to(device),
                   src_tensor_list_assignmentindices.to(device),
                   tgt_tensor_list_assignmentindices.to(device),                         
                                                ) #[numgroups x 2]
    netout = netout[:,:,0,0]
    
    
    src_DA_lable_map= torch.ones([src_DA_output.shape[0], src_DA_output.shape[2],
                                            src_DA_output.shape[3]]).long().cuda()
    src_DA_prob = F.log_softmax(src_DA_output, dim=1)
    src_DA_local_loss = F.nll_loss(src_DA_prob, src_DA_lable_map, reduce=False)
    src_DA_local_loss = src_DA_local_loss.mean()
    
    tgt_DA_lable_map= torch.zeros([tgt_DA_output.shape[0], tgt_DA_output.shape[2],
                                            tgt_DA_output.shape[3]]).long().cuda()
    tgt_DA_prob = F.log_softmax(tgt_DA_output, dim=1)
    tgt_DA_local_loss = F.nll_loss(tgt_DA_prob, tgt_DA_lable_map, reduce=False)
    tgt_DA_local_loss = tgt_DA_local_loss.mean()
    
    
    
    src_global_DA_lable_map= torch.ones([src_global_DA_output.shape[0], src_global_DA_output.shape[2],
                                            src_global_DA_output.shape[3]]).long().cuda()
    src_global_DA_prob = F.log_softmax(src_global_DA_output, dim=1)
    src_global_DA_loss = F.nll_loss(src_global_DA_prob, src_global_DA_lable_map, reduce=False)
    src_global_DA_loss = src_global_DA_loss.mean()
    
    tgt_global_DA_lable_map= torch.zeros([tgt_global_DA_output.shape[0], tgt_global_DA_output.shape[2],
                                            tgt_global_DA_output.shape[3]]).long().cuda()
    tgt_global_DA_prob = F.log_softmax(tgt_global_DA_output, dim=1)
    tgt_global_DA_loss = F.nll_loss(tgt_global_DA_prob, tgt_global_DA_lable_map, reduce=False)
    tgt_global_DA_loss = tgt_global_DA_loss.mean()
    

    
    loss = (criterion(netout, src_y.cuda()) + 0.1* (src_DA_local_loss+tgt_DA_local_loss) + 0.1 *(src_global_DA_loss+tgt_global_DA_loss) )/(stepsize_gradupdate+0.0)
    
    
    loss.backward()
    
    if((batchcount%10) == 0):
        print(" loss = {}".format(loss.item()))
    
    if((batchcount%stepsize_gradupdate)==0):
        optimizer.step()
        optimizer.zero_grad()
    history_trainingloss.append(loss.detach().cpu().numpy())

        
    if( batchcount == 30000 or batchcount == 40000 or batchcount == 50000):
        
        str_now = 'src_' + str(batchcount)
            
        #save
        torch.save(model.state_dict(),"Trained_Models/model_{}.pt".format(str_now))
        #save metainf =======
        kwargs_getdataset = {
            "idx_split":idx_split,
            "scale_foreground":scale_thumbnail
        }
        kwargs_model = {
            "num_classes":num_classes,\
            "num_visualwords":num_visualwords,\
            "device_stg3":device,\
            "size_input":[batchsize,3,224,224]
        }
        kwargs_optimizer = {
            "type":optimizer.__class__.__name__,
            "lr":lr_optimizer
        }
        traininginfo = {
            "idx_split":1,
            "val_randomseed":15,
            "batchsize":32,
            "stepsize_gradupdate":20,
            "num_warmup_droppedpatches":400,\
            "num_fedbatches":len(history_trainingloss)
        }
        TODUMP = {
            "kwargs_getdataset":kwargs_getdataset,
            "kwargs_model":kwargs_model,
            "kwargs_optimizer":kwargs_optimizer,
            "traininginfo":traininginfo
        }
        pickle.dump(TODUMP,
                    open(
                         "Trained_Models/metainf_{}.pkl"\
                         .format(str_now), "wb"
                      )
                )
    if(batchcount > 50050):
        src_train_dl.pause_loading()
        tgt_train_dl.pause_loading()
        break

    
print("elpased time = {} seconds".format(time.time()-tstart))


    the batch dropped (warm-up).
    the batch dropped (warm-up).
    the batch dropped (warm-up).
    the batch dropped (warm-up).
    the batch dropped (warm-up).
    the batch dropped (warm-up).
    the batch dropped (warm-up).
    the batch dropped (warm-up).
    the batch dropped (warm-up).
 batchcount 10


/home/microscopy-imaging/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


 loss = 0.9215044975280762
 batchcount 20
 loss = 0.9231885075569153
 batchcount 30
 loss = 0.8830077052116394
 batchcount 40
 loss = 0.9339562654495239
 batchcount 50
 loss = 0.8265867233276367
 batchcount 60
 loss = 0.9160025715827942
 batchcount 70
 loss = 0.8163451552391052
 batchcount 80
 loss = 0.7212371230125427
 batchcount 90
 loss = 0.9104688763618469
 batchcount 100
 loss = 0.8140446543693542
 batchcount 110
 loss = 1.1817951202392578
 batchcount 120
 loss = 1.326983094215393
 batchcount 130
 loss = 1.2093671560287476
 batchcount 140
 loss = 0.9509997367858887
 batchcount 150
 loss = 0.8883240818977356
 batchcount 160
 loss = 0.6966122388839722
 batchcount 170
 loss = 0.7775923609733582
 batchcount 180
 loss = 0.9157611131668091
 batchcount 190
 loss = 0.7124612927436829
 batchcount 200
 loss = 0.6689571142196655
 batchcount 210
 loss = 0.6484362483024597
 batchcount 220
 loss = 0.851966381072998
 batchcount 230
 loss = 0.46271663904190063
 batchcount 240
 loss = 0.6553350090

 batchcount 1880
 loss = 0.08704695850610733
 batchcount 1890
 loss = 0.08759339898824692
 batchcount 1900
 loss = 0.12770803272724152
 batchcount 1910
 loss = 0.1593347042798996
 batchcount 1920
 loss = 0.16743990778923035
 batchcount 1930
 loss = 0.17959004640579224
 batchcount 1940
 loss = 0.11128871887922287
 batchcount 1950
 loss = 0.05902485176920891
 batchcount 1960
 loss = 0.03084900788962841
 batchcount 1970
 loss = 0.07961650937795639
 batchcount 1980
 loss = 0.07885449379682541
 batchcount 1990
 loss = 0.07354814559221268
 batchcount 2000
 loss = 0.0826224610209465
 batchcount 2010
 loss = 0.12979966402053833
 batchcount 2020
 loss = 0.17587678134441376
 batchcount 2030
 loss = 0.11624767631292343
 batchcount 2040
 loss = 0.17228959500789642
 batchcount 2050
 loss = 0.33723562955856323
 batchcount 2060
 loss = 0.08755727857351303
 batchcount 2070
 loss = 0.10656153410673141
 batchcount 2080
 loss = 0.11133908480405807
 batchcount 2090
 loss = 0.10834046453237534
 batchcount 

 batchcount 3710
 loss = 0.05946556478738785
 batchcount 3720
 loss = 0.10977987200021744
 batchcount 3730
 loss = 0.06974375247955322
 batchcount 3740
 loss = 0.09964660555124283
 batchcount 3750
 loss = 0.06185287982225418
 batchcount 3760
 loss = 0.14611928164958954
 batchcount 3770
 loss = 0.06789388507604599
 batchcount 3780
 loss = 0.07362540811300278
 batchcount 3790
 loss = 0.09114038199186325
 batchcount 3800
 loss = 0.09497936815023422
 batchcount 3810
 loss = 0.06008227542042732
 batchcount 3820
 loss = 0.0875953733921051
 batchcount 3830
 loss = 0.035634350031614304
 batchcount 3840
 loss = 0.07820355892181396
 batchcount 3850
 loss = 0.0972689762711525
 batchcount 3860
 loss = 0.1705293506383896
 batchcount 3870
 loss = 0.11299856007099152
 batchcount 3880
 loss = 0.1545628309249878
 batchcount 3890
 loss = 0.0852283164858818
 batchcount 3900
 loss = 0.13046006858348846
 batchcount 3910
 loss = 0.06824218481779099
 batchcount 3920
 loss = 0.08492133021354675
 batchcount 39

...

 batchcount 42970
 loss = 0.014204044826328754
 batchcount 42980
 loss = 0.027489835396409035
 batchcount 42990
 loss = 0.028921008110046387
 batchcount 43000
 loss = 0.05433112010359764
 batchcount 43010
 loss = 0.020062580704689026
 batchcount 43020
 loss = 0.04032846540212631
 batchcount 43030
 loss = 0.06515957415103912
 batchcount 43040
 loss = 0.025694197043776512
 batchcount 43050
 loss = 0.0405830517411232
 batchcount 43060
 loss = 0.037142809480428696
 batchcount 43070
 loss = 0.0338096097111702
 batchcount 43080
 loss = 0.04596630856394768
 batchcount 43090
 loss = 0.044470831751823425
 batchcount 43100
 loss = 0.038826823234558105
 batchcount 43110
 loss = 0.022587528452277184
 batchcount 43120
 loss = 0.009307409636676311
 batchcount 43130
 loss = 0.02967703901231289
 batchcount 43140
 loss = 0.030677367001771927
 batchcount 43150
 loss = 0.05986647680401802
 batchcount 43160
 loss = 0.0522286556661129
 batchcount 43170
 loss = 0.055441487580537796
 batchcount 43180
 loss =

 loss = 0.033613573759794235
 batchcount 44750
 loss = 0.06042015552520752
 batchcount 44760
 loss = 0.03582300990819931
 batchcount 44770
 loss = 0.10142935812473297
 batchcount 44780
 loss = 0.08352877199649811
 batchcount 44790
 loss = 0.06662110984325409
 batchcount 44800
 loss = 0.05467468500137329
 batchcount 44810
 loss = 0.05818336084485054
 batchcount 44820
 loss = 0.06301485747098923
 batchcount 44830
 loss = 0.03482739254832268
 batchcount 44840
 loss = 0.04943118989467621
 batchcount 44850
 loss = 0.10383160412311554
 batchcount 44860
 loss = 0.1483563780784607
 batchcount 44870
 loss = 0.17782633006572723
 batchcount 44880
 loss = 0.05468923598527908
 batchcount 44890
 loss = 0.06038864329457283
 batchcount 44900
 loss = 0.045226652175188065
 batchcount 44910
 loss = 0.0604136660695076
 batchcount 44920
 loss = 0.03487548604607582
 batchcount 44930
 loss = 0.05542704090476036
 batchcount 44940
 loss = 0.06705360859632492
 batchcount 44950
 loss = 0.03347970172762871
 batch

 loss = 0.01933814026415348
 batchcount 46520
 loss = 0.03454066067934036
 batchcount 46530
 loss = 0.037417229264974594
 batchcount 46540
 loss = 0.043849337846040726
 batchcount 46550
 loss = 0.05234193429350853
 batchcount 46560
 loss = 0.06949575990438461
 batchcount 46570
 loss = 0.02814660407602787
 batchcount 46580
 loss = 0.08551400154829025
 batchcount 46590
 loss = 0.08999644964933395
 batchcount 46600
 loss = 0.014993421733379364
 batchcount 46610
 loss = 0.017980242148041725
 batchcount 46620
 loss = 0.059197377413511276
 batchcount 46630
 loss = 0.10045384615659714
 batchcount 46640
 loss = 0.05606867000460625
 batchcount 46650
 loss = 0.05671817064285278
 batchcount 46660
 loss = 0.08443602919578552
 batchcount 46670
 loss = 0.0373845249414444
 batchcount 46680
 loss = 0.030840778723359108
 batchcount 46690
 loss = 0.04682162031531334
 batchcount 46700
 loss = 0.022587208077311516
 batchcount 46710
 loss = 0.04626476764678955
 batchcount 46720
 loss = 0.011632916517555714

 loss = 0.04238488897681236
 batchcount 48290
 loss = 0.010882922448217869
 batchcount 48300
 loss = 0.051429856568574905
 batchcount 48310
 loss = 0.039041999727487564
 batchcount 48320
 loss = 0.06030462309718132
 batchcount 48330
 loss = 0.04726145789027214
 batchcount 48340
 loss = 0.04295488819479942
 batchcount 48350
 loss = 0.06474564969539642
 batchcount 48360
 loss = 0.0735832154750824
 batchcount 48370
 loss = 0.07393597811460495
 batchcount 48380
 loss = 0.038097407668828964
 batchcount 48390
 loss = 0.04295865818858147
 batchcount 48400
 loss = 0.04506044089794159
 batchcount 48410
 loss = 0.1309988647699356
 batchcount 48420
 loss = 0.022270724177360535
 batchcount 48430
 loss = 0.024557625874876976
 batchcount 48440
 loss = 0.021421417593955994
 batchcount 48450
 loss = 0.0578535795211792
 batchcount 48460
 loss = 0.06778600066900253
 batchcount 48470
 loss = 0.08748126029968262
 batchcount 48480
 loss = 0.015356853604316711
 batchcount 48490
 loss = 0.0625990778207779
 b

 loss = 0.03634413704276085
elpased time = 102880.38539028168 seconds


In [8]:
str_now


'src_50000'